### Visualizing the BIOS

In [ ]:
from dataset import SINR_DS
# TODO: The call on SINR_DS needs to be updated

PREDICTORS = "loc_env_sent2"
PREDICTORS = "loc_env"
#PREDICTORS = "loc_env_LR"
#PREDICTORS = "loc"
BATCHSIZE = 512
BATCHSIZE = 2048
# Low Resolution: 
if PREDICTORS.endswith("LR"):
    bioclim_path = "/data/jdolli/glc23_data/sinr_data/data/env/bioclim_elevation_scaled_europe.npy"
# High Resolution:
else:
    bioclim_path = "/data/jdolli/bioclim+elev/bioclim_elevation_scaled_europe.npy"
USV = False
dataset = SINR_DS(dataset_file, PREDICTORS, bioclim_path = bioclim_path, use_subm_val=USV)
ds_train, ds_val = torch.utils.data.random_split(dataset, [0.9, 0.1])
train_loader = torch.utils.data.DataLoader(ds_train, batch_size=BATCHSIZE, num_workers=16)
val_loader = torch.utils.data.DataLoader(ds_val, batch_size=BATCHSIZE, num_workers=16)

In [ ]:
max_lon = 34.55792
min_lon = -10.53904
max_lat = 71.18392
min_lat = 34.56858

RES_LON = 502
RES_LAT = 408

from tqdm import tqdm
# For Sent-Sinr
preds = []
for i in tqdm(range(RES_LON)):
    # i is lon
    # j is lat
    for j in range(RES_LAT):
        lon = i/RES_LON
        lat = j/RES_LAT
        lon = lon * (max_lon - min_lon) + min_lon
        lat = lat * (max_lat - min_lat) + min_lat
        loc = dataset.encode(lon, lat)
        preds.append(loc[4:])
preds = torch.stack(preds)
print(preds.shape)

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image
import numpy as np
import os

bioses = os.listdir("/data/jdolli/bioclim+elev")

for sid in range(20):
    
    #vmin = min(preds[:, sid].min(), 1 - preds[:, sid].max())
    #vmax = max(preds[:, sid].max(), 1 - preds[:, sid].min())
    
    vmin = preds[:, sid].min()
    vmax = preds[:, sid].max()

    mask = np.load(os.path.join("/data/jdolli/glc23_data/sinr_data/data/masks", 'ocean_mask_hr.npy'))
    lon_res = mask.shape[1] / 360
    lat_res = mask.shape[0] / 180
    north = int((90-max_lat) * lat_res)
    south = int((90-min_lat) * lat_res)
    west = int((180 + min_lon) * lon_res)
    east = int((180 + max_lon) * lon_res)
    mask = mask[north:south, west:east]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_xlim([-10.53904, 34.55792])
    ax.set_ylim([34.56858, 71.18392])
    cmap = plt.cm.plasma
    cmap.set_bad(color='none')
    mask_inds = np.where(mask.reshape(-1) == 1)[0]
    
    im = preds[:, sid]
    print("Bios:", bioses[sid], im.min().item(), im.max().item())
    im = torch.rot90(im.view(RES_LON, RES_LAT))
    im = torch.reshape(im, (RES_LAT * RES_LON, 1))
    im = im[mask_inds]
    
    op_im = np.ones(mask.shape[0] * mask.shape[1]) * np.nan
    op_im[mask_inds] = im.detach().view(len(mask_inds)).numpy()
    op_im = np.ma.masked_invalid(op_im)
    op_im = op_im.reshape(RES_LAT, RES_LON)
    
    TRESHHOLD = 0
    if TRESHHOLD > 0:
        #op_im[op_im > TRESHHOLD] = 1
        op_im[op_im <= TRESHHOLD] = 0
    
    im = ax.imshow(op_im, extent=(-10.53904, 34.55792, 34.56858, 71.18392), vmin=vmin, vmax=vmax, cmap=cmap)
    
    fig.colorbar(im, ax=ax)
    
    plt.show()
    

# Create distribution graphic

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
#dataset_file = pd.read_csv('/data/jdolli/Presences_only_train.csv', sep=";", header='infer', low_memory=False)
dataset_file = pd.read_csv('/data/jdolli/Pot_10_to_1000.csv', sep=";", header='infer', low_memory=False)

In [ ]:
ids = dataset_file["speciesId"].to_numpy()

In [ ]:
counter = Counter(ids)

In [ ]:
amounts = counter.values()
amounts = list(amounts)
amounts.sort(reverse=True)
print(len(amounts))

In [ ]:
fig = plt.figure(figsize = (10, 5))
plt.bar([i for i in range(len(amounts))], amounts)

In [ ]:
fig.savefig("./class_distribution.png")